# Halimbawa 01: Mabilisang Chat gamit ang OpenAI SDK

Ipinapakita ng notebook na ito kung paano gamitin ang OpenAI SDK kasama ang Microsoft Foundry Local para sa mabilisang pakikipag-chat.

## Pangkalahatang-ideya

Ipinapakita ng halimbawang ito:
- Paggamit ng OpenAI Python SDK kasama ang Foundry Local
- Pamamahala sa parehong Azure OpenAI at lokal na Foundry na mga configuration
- Pagpapatupad ng tamang paghawak ng error at mga fallback na estratehiya
- Paggamit ng FoundryLocalManager para sa pamamahala ng serbisyo


## Mga Paunang Kailangan

Siguraduhing mayroon ka ng:
1. Nakainstall at gumagana ang Foundry Local
2. Isang modelong naka-load (hal., `foundry model run phi-4-mini`)
3. Mga kinakailangang Python packages na naka-install

### I-install ang mga Kinakailangan


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## I-import ang Mga Kinakailangang Aklatan


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Konpigurasyon

I-set up ang iyong konpigurasyon. Maaari kang gumamit ng alinman sa:
1. **Azure OpenAI** (batay sa ulap)
2. **Foundry Local** (lokal na inference)

### Opsyon 1: Konpigurasyon ng Azure OpenAI


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Opsyon 2: Lokal na Konpigurasyon ng Foundry


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Pagsisimula ng Kliyente

Gumawa ng OpenAI client batay sa iyong configuration:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Halimbawa ng Pangunahing Chat

Subukan natin ang isang simpleng pakikipag-usap:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Interaktibong Sesyon ng Chat

Subukan ang iba't ibang uri ng mga prompt upang makita kung paano tumutugon ang modelo:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Advanced Usage: Pag-stream ng mga Tugon

Para sa mas mahahabang tugon, ang pag-stream ay maaaring magbigay ng mas magandang karanasan sa gumagamit:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Paghawak ng Error at Diagnostiko

Tingnan natin ang status ng serbisyo at mga available na modelo:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Pagsubok ng Custom na Prompt

Gamitin ang cell sa ibaba upang subukan ang sarili mong mga prompt:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Buod

Ipinakita ng notebook na ito:

1. **✅ Pag-set up ng Client**: Paano i-configure ang OpenAI SDK gamit ang Foundry Local
2. **✅ Pangunahing Chat**: Simpleng pakikipag-ugnayan ng request-response
3. **✅ Streaming**: Pagbuo ng tugon sa real-time
4. **✅ Pag-aayos ng Error**: Matibay na paghawak ng error at diagnostics
5. **✅ Kalusugan ng Serbisyo**: Pagsusuri ng availability ng modelo at status ng serbisyo

### Mga Susunod na Hakbang

- **Sample 02**: Advanced SDK integration na may suporta sa Azure OpenAI
- **Sample 04**: Pagbuo ng Chainlit chat application
- **Sample 05**: Mga sistema ng multi-agent orchestration
- **Sample 06**: Matalinong pag-route ng modelo

### Mga Pangunahing Benepisyo ng Foundry Local

- 🔒 **Privacy**: Hindi lumalabas ang data sa iyong device
- ⚡ **Bilis**: Mababang latency sa lokal na inference
- 💰 **Gastos**: Walang gastos sa paggamit ng API
- 🔌 **Offline**: Gumagana kahit walang koneksyon sa internet
- 🛠️ **Compatibility**: API na compatible sa OpenAI
